In [64]:
import tensorflow as tf
from keras import layers
from keras import activations
from keras import losses
from keras import optimizers
from keras import metrics
from keras.datasets import mnist

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [65]:
train_dataset = pd.read_csv('./dataset/titanic_train.csv')
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [66]:
train_dataset.replace(['female', 'male'], [0, 1], inplace=True)
train_dataset.fillna(0, inplace=True)
train_dataset.head()

/tmp/ipykernel_6647/4199206752.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_dataset.replace(['female', 'male'], [0, 1], inplace=True)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,S


In [67]:
train_dataset.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
train_dataset.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,22.0,1,0,7.2500
1,2,1,1,0,38.0,1,0,71.2833
2,3,1,3,0,26.0,0,0,7.9250
3,4,1,1,0,35.0,1,0,53.1000
4,5,0,3,1,35.0,0,0,8.0500


In [68]:
train_dataset.corr()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.038125,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.543351,0.010539,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.361353,0.083081,0.018443,-0.549500
Sex,0.042939,-0.543351,0.131900,1.000000,0.024978,-0.114631,-0.245489,-0.182333
Age,0.038125,0.010539,-0.361353,0.024978,1.000000,-0.184664,-0.048786,0.135516
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.184664,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.048786,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,-0.182333,0.135516,0.159651,0.216225,1.000000


In [69]:
x = train_dataset[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']].values
y = train_dataset[['Survived']].values

x.shape, y.shape

((891, 6), (891, 1))

In [70]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((712, 6), (179, 6), (712, 1), (179, 1))

In [71]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10).batch(6)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(6)

In [72]:
class Classifier(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.dense1 = layers.Dense(6, activation=activations.relu)
        self.dense2 = layers.Dense(64, activation=activations.relu)
        self.dense3 = layers.Dense(32, activation=activations.relu)
        self.dense4 = layers.Dense(2, activation=activations.softmax)

        self.loss_func = losses.SparseCategoricalCrossentropy(True)
        self.optimizer = optimizers.Adam()

        self.train_loss = metrics.Mean()
        self.test_loss = metrics.Mean()

        self.train_accuracy = metrics.SparseCategoricalAccuracy()
        self.test_accuracy = metrics.SparseCategoricalAccuracy()

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        
        return self.dense4(x)

    def build_model(self):
        self.model = Classifier()

    @tf.function
    def train(self, images, labels):
        # tape: tf.GradientTape
        with tf.GradientTape() as tape:
            pred = self.model(images)
            loss = self.loss_func(labels, pred)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_variables))

        self.train_loss(loss)
        self.train_accuracy(labels, pred)

    @tf.function
    def test(self, images, labels):
        pred = self.model(images)
        loss = self.loss_func(labels, pred)

        self.test_loss(loss)
        self.test_accuracy(labels, pred)

    def fit(self, train_dataset, test_dataset, epochs):
        for epoch in range(1, epochs + 1):
            self.train_loss.reset_states()
            self.train_accuracy.reset_states()
            self.test_loss.reset_states()
            self.test_accuracy.reset_states()

            for imgs, labels in train_dataset:
                self.train(imgs, labels)

            for imgs, labels in test_dataset:
                self.test(imgs, labels)

            show_train_loss = round(float(self.train_loss.result()), 4)
            show_train_acc = round(float(self.train_accuracy.result()), 4)
            show_test_loss = round(float(self.test_loss.result()), 4)
            show_test_acc = round(float(self.test_accuracy.result()), 4)
            print(f'Epoch: {epoch}, Train Loss: {show_train_loss}, Train Accuracy: {show_train_acc}, ' +
                  f'Test Loss: {show_test_loss}, Test Accuracy: {show_test_acc}')

    def load_model(self, model_path):
        self.model = tf.keras.models.load_model(model_path)

In [74]:
model = Classifier()
model.build_model()
model.fit(train_dataset, test_dataset, 350)

Epoch: 1, Train Loss: 0.7607, Train Accuracy: 0.6053, Test Loss: 0.6619, Test Accuracy: 0.6872
Epoch: 2, Train Loss: 0.7149, Train Accuracy: 0.6433, Test Loss: 0.694, Test Accuracy: 0.6816
Epoch: 3, Train Loss: 0.6809, Train Accuracy: 0.6615, Test Loss: 0.6651, Test Accuracy: 0.6704
Epoch: 4, Train Loss: 0.7122, Train Accuracy: 0.6713, Test Loss: 0.634, Test Accuracy: 0.6592
Epoch: 5, Train Loss: 0.6531, Train Accuracy: 0.6742, Test Loss: 0.6135, Test Accuracy: 0.6704
Epoch: 6, Train Loss: 0.6367, Train Accuracy: 0.6756, Test Loss: 0.6176, Test Accuracy: 0.6592
Epoch: 7, Train Loss: 0.6107, Train Accuracy: 0.684, Test Loss: 0.613, Test Accuracy: 0.6704
Epoch: 8, Train Loss: 0.5989, Train Accuracy: 0.6966, Test Loss: 0.5909, Test Accuracy: 0.6704
Epoch: 9, Train Loss: 0.5957, Train Accuracy: 0.6938, Test Loss: 0.5801, Test Accuracy: 0.6816
Epoch: 10, Train Loss: 0.5693, Train Accuracy: 0.7177, Test Loss: 0.6054, Test Accuracy: 0.6872
Epoch: 11, Train Loss: 0.5652, Train Accuracy: 0.7191